In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the images to [0, 1] range
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Step 2: Build the model with specified layers
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flattens the 28x28 image to 784 inputs
    Dense(50, activation='relu'),   # First hidden layer with 50 neurons
    Dense(20, activation='relu'),   # Second hidden layer with 20 neurons
    Dense(10, activation='relu'),   # Third hidden layer with 10 neurons
    Dense(10, activation='softmax') # Output layer with 10 neurons (one for each class)
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 3: Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=32, validation_split=0.1)

# Step 4: Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.4f}")

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/5
1688/1688 [==============================] - 9s 4ms/step - loss: 0.4069 - accuracy: 0.8748 - val_loss: 0.1725 - val_accuracy: 0.9492
Epoch 2/5
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1733 - accuracy: 0.9498 - val_loss: 0.1240 - val_accuracy: 0.9642
Epoch 3/5
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1329 - accuracy: 0.9613 - val_loss: 0.1156 - val_accuracy: 0.9627
Epoch 4/5
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1101 - accuracy: 0.9669 - val_loss: 0.1145 - val_accuracy: 0.9658
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.1256 - accuracy: 0.9649
Test accuracy: 0.9649


In [3]:
#Save the entire model in model.h5 file
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()#saving converted model in "converted_model.tflite" file
open("converted_model.tflite", "wb").write(tflite_model)

165156

In [5]:
mnist_train, _ = tf.keras.datasets.mnist.load_data()
images = tf.cast(mnist_train[0], tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

In [6]:
model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.representative_dataset = representative_data_gen
tflite_quant_model = converter.convert()#saving converted model in "converted_quant_model.tflite" file
open("converted_quant_model.tflite", "wb").write(tflite_quant_model)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


44512

In [7]:
import os
print("Float model in Mb:", os.path.getsize('converted_model.tflite') / float(2**20))
print("Quantized model in Mb:", os.path.getsize('converted_quant_model.tflite') / float(2**20))
print("Compression ratio:", os.path.getsize('converted_model.tflite')/os.path.getsize('converted_quant_model.tflite'))

Float model in Mb: 0.15750503540039062
Quantized model in Mb: 0.042449951171875
Compression ratio: 3.7103702372393963


In [8]:
# Load MNIST test data
(_, _), (test_images, test_labels) = mnist.load_data()
test_images = test_images / 255.0  # Normalize the images

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="converted_quant_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [9]:
# Function to run inference on a single image
def run_inference(image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

In [10]:
# Evaluate accuracy
correct_predictions = 0
for i in range(len(test_images)):
    # Prepare the image for the model based on the model's input shape
    image = np.expand_dims(test_images[i], axis=-1)  # Add channel dimension
    image = np.expand_dims(image, axis=0).astype(np.float32)  # Add batch dimension

    # Check if we need to adjust dimensions to fit the model's expected input
    expected_shape = input_details[0]['shape']
    if len(expected_shape) == 3:  # If model expects (1, height, width)
        image = image.squeeze(axis=-1)  # Remove the channel dimension

    predicted = run_inference(image)
    if np.argmax(predicted) == test_labels[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(test_images)
print(f"Quantized Model Test Accuracy: {accuracy:.4f}")

Quantized Model Test Accuracy: 0.9641


In [11]:
# Path to your quantized TFLite model
tflite_model_path = "converted_quant_model.tflite"

# Load the TFLite model using the Interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get details for all tensors
tensor_details = interpreter.get_tensor_details()

In [12]:
# Iterate over each tensor
for detail in tensor_details:
    print("Tensor Name:", detail['name'])
    # Identify if the tensor is a weight or bias
    if 'kernel' in detail['name'] or 'bias' in detail['name']:
        # Retrieve the tensor data (weights/biases) using the tensor index
        tensor = interpreter.get_tensor(detail['index'])

        # Print the layer name and tensor details
        print(f"Layer: {detail['name']}")
        print("Shape:", detail['shape'])

        # Check if the tensor has quantization parameters
        if detail['quantization_parameters']:
            print("Quantized Scale:", detail['quantization_parameters']['scales'])
            print("Quantized Zero Point:", detail['quantization_parameters']['zero_points'])

        print("Tensor Values:")
        print(tensor)
        print("\n")

Tensor Name: serving_default_flatten_input:0
Tensor Name: sequential/flatten/Const
Tensor Name: sequential/dense_3/BiasAdd/ReadVariableOp
Tensor Name: sequential/dense_3/MatMul
Tensor Name: sequential/dense_2/BiasAdd/ReadVariableOp
Tensor Name: sequential/dense_2/MatMul
Tensor Name: sequential/dense_1/BiasAdd/ReadVariableOp
Tensor Name: sequential/dense_1/MatMul
Tensor Name: sequential/dense/BiasAdd/ReadVariableOp
Tensor Name: sequential/dense/MatMul
Tensor Name: tfl.quantize
Tensor Name: sequential/flatten/Reshape
Tensor Name: sequential/dense/MatMul;sequential/dense/Relu;sequential/dense/BiasAdd
Tensor Name: sequential/dense_1/MatMul;sequential/dense_1/Relu;sequential/dense_1/BiasAdd
Tensor Name: sequential/dense_2/MatMul;sequential/dense_2/Relu;sequential/dense_2/BiasAdd
Tensor Name: sequential/dense_3/MatMul;sequential/dense_3/BiasAdd
Tensor Name: StatefulPartitionedCall:01
Tensor Name: StatefulPartitionedCall:0


In [13]:
# Iterate over each tensor
for detail in tensor_details:
    # Identify if the tensor is likely a weight or bias
    if 'MatMul' in detail['name'] or 'BiasAdd' in detail['name']:
        # Retrieve the tensor data using the tensor index
        tensor = interpreter.get_tensor(detail['index'])

        # Print the layer name and tensor details
        print(f"Layer: {detail['name']}")
        print("Shape:", detail['shape'])

        # Check if the tensor has quantization parameters
        if detail['quantization_parameters']:
            print("Quantized Scale:", detail['quantization_parameters']['scales'])
            print("Quantized Zero Point:", detail['quantization_parameters']['zero_points'])

        print("Tensor Values:")
        print(tensor)
        print("\n")

Layer: sequential/dense_3/BiasAdd/ReadVariableOp
Shape: [10]
Quantized Scale: [0.00072551]
Quantized Zero Point: [0]
Tensor Values:
[-354  -38  123 -337  418  316  -25 -250  -20   44]


Layer: sequential/dense_3/MatMul
Shape: [10 10]
Quantized Scale: [0.00735435]
Quantized Zero Point: [0]
Tensor Values:
[[ 107   -7   37  -54   83   -7  -75  -32   58   39]
 [  92   26  -80  -53  -42   55   42   95  -52  -56]
 [ -60  -14   68  -60   26   25  111  -97   -4  -76]
 [ -48   53  -75   15   31   38    5   28   58  -41]
 [ -16 -127    6    6   22   35  -43  -24  -89  121]
 [ -39   67   28   83  -16 -108   17   66  -15  -69]
 [  85  -69  -16   48   13 -107   76   58  -61   -5]
 [ -64  112   21  -64   19   51   15  -14 -104   18]
 [ -68  -64   67  -29  -21   21   14  110    5  -64]
 [  26  -58   12   78   40   73  -77  -92    0  -73]]


Layer: sequential/dense_2/BiasAdd/ReadVariableOp
Shape: [10]
Quantized Scale: [0.00057236]
Quantized Zero Point: [0]
Tensor Values:
[ -56  222  202  333   36 -162

ValueError: Tensor data is null. Run allocate_tensors() first

In [ ]:
# Path to your quantized TFLite model
tflite_model_path = "converted_quant_model.tflite"

# Load the TFLite model using the Interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()  # Ensure tensors are allocated

# Get details for all tensors
tensor_details = interpreter.get_tensor_details()

# Open a file to write the weights and biases
with open("model_weights_and_biases.txt", "w") as file:
    # Iterate over each tensor
    for detail in tensor_details:
        # Identify if the tensor is likely a weight or bias
        if 'MatMul' in detail['name'] or 'BiasAdd' in detail['name']:
            # Retrieve the tensor data using the tensor index
            tensor = interpreter.get_tensor(detail['index'])

            # Write the layer name and tensor details to the file
            file.write(f"Layer: {detail['name']}\n")
            file.write("Shape: {}\n".format(detail['shape']))

            # Check if the tensor has quantization parameters
            if detail['quantization_parameters']:
                file.write("Quantized Scale: {}\n".format(detail['quantization_parameters']['scales']))
                file.write("Quantized Zero Point: {}\n".format(detail['quantization_parameters']['zero_points']))

            file.write("Tensor Values:\n")
            file.write("{}\n\n".format(tensor))


ValueError: Tensor data is null. Run allocate_tensors() first

In [ ]:
# Path to your quantized TFLite model
tflite_model_path = "converted_quant_model.tflite"

# Load the TFLite model using the Interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()  # Ensure tensors are allocated

# Get details for all tensors
tensor_details = interpreter.get_tensor_details()

# Prepare to display full arrays without truncation
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # Avoid truncation

# Extract and print the 'sequential/dense/MatMul' layer specifically
for detail in tensor_details:
    if detail['name'] == 'sequential/dense/MatMul':
        # Access the tensor data
        interpreter.allocate_tensors()  # Ensure valid state
        tensor = interpreter.tensor(detail['index'])()  # Get tensor data

        # Prepare the display content
        content = [
            f"Layer: {detail['name']}",
            f"Shape: {detail['shape']}",
            f"Quantized Scale: {detail['quantization_parameters']['scales']}",
            f"Quantized Zero Point: {detail['quantization_parameters']['zero_points']}",
            "Tensor Values:",
            f"{tensor}"
        ]

        # Combine all parts into a single output
        full_details = "\n".join(content)

        # Print or save this information as needed
        print(full_details)
        break  # Since we're only interested in this specific layer, we can stop after finding it

Layer: sequential/dense/MatMul
Shape: [ 50 784]
Quantized Scale: [0.00692134]
Quantized Zero Point: [0]
Tensor Values:
[[  -1    1    0   -1    7    3    9    1    1    8   10    8   -8    4   18   -5   -1   10   -3   -7    0   -7    9  -10  -12   -7   10   11   -7    3   10    3    3    4   20   32   34    5   12   13    8    7   29   33   26    7    8   21   18   -8   19   20   -1    3    2  -11    0  -10    2    9    4   16   23   33   28   13   19    5    6   44   23   44   30   45   50   28   20   -5  -10    9    2   20    7   -4   -4   11   -9   18  -12  -11   -4    3  -15   16  -12  -28  -16    7    4   29    5  -12    3   14   25   12   13   -6    6    5   -6    6   -2  -13   -9   -9   -1   17   24   11   -3  -19    9    1  -13  -12   13    3    9    0   14   18   -4    4   -6   25   33   11    5    7   -6    6   -3  -28  -10   -2   -1  -25  -20    7   21   11   -9    2   12   12   22   33   32    7   17   19    2   26   12   -6   12    5   12  -11  -17  -15    1   -8    3  -10

In [15]:

# Path to your quantized TFLite model
tflite_model_path = "converted_quant_model.tflite"

# Load the TFLite model using the Interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()  # Ensure tensors are allocated

# Get details for all tensors
tensor_details = interpreter.get_tensor_details()

# Prepare to display full arrays without truncation
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # Avoid truncation

# Open a file to write the specific layer's weights and biases
with open("specific_layer_weights_and_biases.txt", "w") as file:
    # Extract and write the 'sequential/dense/MatMul' layer specifically
    for detail in tensor_details:
        if detail['name'] == 'sequential/dense/MatMul':
            # Access the tensor data
            interpreter.allocate_tensors()  # Ensure valid state
            tensor = interpreter.tensor(detail['index'])()  # Get tensor data

            # Write the layer details to the file
            file.write(f"Layer: {detail['name']}\n")
            file.write(f"Shape: {detail['shape']}\n")
            file.write(f"Quantized Scale: {detail['quantization_parameters']['scales']}\n")
            file.write(f"Quantized Zero Point: {detail['quantization_parameters']['zero_points']}\n")
            file.write("Tensor Values:\n")
            file.write(f"{tensor}\n\n")

            print("Saved specific layer details to specific_layer_weights_and_biases.txt")
            break  # Since only this layer's details are needed, stop after processing it

Saved specific layer details to specific_layer_weights_and_biases.txt


In [16]:
import numpy as np
import tensorflow as tf

# Path to your quantized TFLite model
tflite_model_path = "converted_quant_model.tflite"

# Load the TFLite model using the Interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()  # Ensure tensors are allocated

# Get details for all tensors
tensor_details = interpreter.get_tensor_details()

# Prepare to display full arrays without truncation
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # Avoid truncation

# Helper function to format numpy arrays as SystemVerilog arrays
def format_sv_weights_array(tensors):
    sv_output = "'{\n"
    for tensor in tensors:
        tensor = tensor.astype(int)  # Ensure the tensor values are integers
        for row in tensor:
            sv_output += "    '{"
            sv_output += ", ".join(map(str, row))
            sv_output += "}',\n"
    sv_output += "}"
    return sv_output

def format_sv_biases_array(tensors):
    sv_output = "'{\n"
    for tensor in tensors:
        tensor = tensor.astype(int)  # Ensure the tensor values are integers
        sv_output += "    '{"
        sv_output += ", ".join(map(str, tensor))
        sv_output += "}',\n"
    sv_output += "}"
    return sv_output

# Initialize network parameters
layers = 4
layer_widths = [754, 50, 20, 10, 10]

# Store weights and biases arrays
weights_arrays = []
biases_arrays = []

# Iterate over each tensor and format for SystemVerilog
for detail in tensor_details:
    try:
        tensor = interpreter.get_tensor(detail['index'])
    except ValueError as e:
        print(f"Skipping tensor {detail['name']} due to error: {e}")
        continue

    if 'MatMul' in detail['name']:
        weights_arrays.append(tensor)
    elif 'BiasAdd' in detail['name']:
        biases_arrays.append(tensor)

# Combine all weights and biases into a single 3D and 2D array, respectively
weights_combined = format_sv_weights_array(weights_arrays)
biases_combined = format_sv_biases_array(biases_arrays)

# Print SystemVerilog parameters
print(f"parameter LAYERS = {layers};")
print(f"parameter int LAYER_WIDTHS[LAYERS+1] = {{", ", ".join(map(str, layer_widths)), "}};\n")

print("parameter int WEIGHTS [][][] = ")
print(weights_combined)
print(";\n")

print("parameter int BIASES [][] = ")
print(biases_combined)
print(";\n")


Skipping tensor tfl.quantize due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/flatten/Reshape due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense/MatMul;sequential/dense/Relu;sequential/dense/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense_1/MatMul;sequential/dense_1/Relu;sequential/dense_1/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense_2/MatMul;sequential/dense_2/Relu;sequential/dense_2/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
parameter LAYERS = 4;
parameter int LAYER_WIDTHS[LAYERS+1] = { 754, 50, 20, 10, 10 }};

parameter int WEIGHTS [][][] = 
'{
    '{107, -7, 37, -54, 83, -7, -75, -32, 58, 39}',
    '{92, 26, -80, -53, -42, 55, 42, 95, -52, -56}',
    '{-60, -14, 68, -60, 26, 25, 111, -97, -4, -76}',
    '{-48, 53, -75, 15, 31, 38, 5, 28, 58, -41}',
   

In [25]:
import numpy as np
import tensorflow as tf

# Path to your quantized TFLite model
tflite_model_path = "converted_quant_model.tflite"
output_file_path = "systemverilog_parameters.sv"

# Load the TFLite model using the Interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()  # Ensure tensors are allocated

# Get details for all tensors
tensor_details = interpreter.get_tensor_details()

# Prepare to display full arrays without truncation
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # Avoid truncation

# Helper function to format numpy arrays as SystemVerilog arrays
def format_sv_weights_array(tensors):
    sv_output = "'{\n"
    for tensor in tensors:
        tensor = tensor.astype(int)  # Ensure the tensor values are integers
        for row in tensor:
            sv_output += "    '{"
            sv_output += ", ".join(map(str, row))
            sv_output += "},\n"
    sv_output += "}"
    return sv_output

def format_sv_biases_array(tensors):
    sv_output = "'{\n"
    for tensor in tensors:
        tensor = tensor.astype(int)  # Ensure the tensor values are integers
        sv_output += "    '{"
        sv_output += ", ".join(map(str, tensor))
        sv_output += "},\n"
    sv_output += "}"
    return sv_output

# Initialize network parameters
layers = 4
layer_widths = [754, 50, 20, 10, 10]

# Store weights and biases arrays
weights_arrays = []
biases_arrays = []

# Iterate over each tensor and format for SystemVerilog
for detail in tensor_details:
    try:
        tensor = interpreter.get_tensor(detail['index'])
    except ValueError as e:
        print(f"Skipping tensor {detail['name']} due to error: {e}")
        continue

    if 'MatMul' in detail['name']:
        weights_arrays.append(tensor)
    elif 'BiasAdd' in detail['name']:
        biases_arrays.append(tensor)

# Combine all weights and biases into a single 3D and 2D array, respectively
weights_combined = format_sv_weights_array(weights_arrays)
biases_combined = format_sv_biases_array(biases_arrays)

# Prepare the content to be written to the file
content = ""
content += f"parameter LAYERS = {layers};\n"
content += f"parameter int LAYER_WIDTHS[LAYERS+1] = {{{', '.join(map(str, layer_widths))}}};\n\n"
content += "parameter int WEIGHTS [][][] = \n"
content += weights_combined
content += ";\n\n"
content += "parameter int BIASES [][] = \n"
content += biases_combined
content += ";\n"

# Write the content to a file
with open(output_file_path, 'w') as file:
    file.write(content)

print(f"SystemVerilog parameters have been written to {output_file_path}")


Skipping tensor tfl.quantize due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/flatten/Reshape due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense/MatMul;sequential/dense/Relu;sequential/dense/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense_1/MatMul;sequential/dense_1/Relu;sequential/dense_1/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense_2/MatMul;sequential/dense_2/Relu;sequential/dense_2/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
SystemVerilog parameters have been written to systemverilog_parameters.sv


In [37]:
import numpy as np
import tensorflow as tf

# Path to your quantized TFLite model
tflite_model_path = "converted_quant_model.tflite"
output_file_path = "systemverilog_parameters.sv"

# Load the TFLite model using the Interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()  # Ensure tensors are allocated

# Get details for all tensors
tensor_details = interpreter.get_tensor_details()

# Reverse the order of tensor details
# tensor_details.reverse()

# Prepare to display full arrays without truncation
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # Avoid truncation

# Helper function to format numpy arrays as SystemVerilog arrays
def format_sv_weights_array(tensors):
    sv_output = "'{\n"
    for tensor in tensors:
        tensor = tensor.astype(int)  # Ensure the tensor values are integers
        sv_output += "    '{"
        for row in tensor:
            sv_output += "'{" + ", ".join(map(str, row)) + "}, "
        sv_output = sv_output.rstrip(", ")  # Remove the last comma and space
        sv_output += "},\n"
    sv_output += "}"
    return sv_output

def format_sv_biases_array(tensors):
    sv_output = "'{\n"
    for tensor in tensors:
        tensor = tensor.astype(int)  # Ensure the tensor values are integers
        sv_output += "    '{"
        sv_output += ", ".join(map(str, tensor))
        sv_output += "},\n"
    sv_output += "}"
    return sv_output

# Initialize network parameters
layers = 4
layer_widths = [754, 50, 20, 10, 10]

# Store weights and biases arrays
weights_arrays = []
biases_arrays = []

# Iterate over each reversed tensor and format for SystemVerilog
for detail in tensor_details:
    try:
        tensor = interpreter.get_tensor(detail['index'])
    except ValueError as e:
        print(f"Skipping tensor {detail['name']} due to error: {e}")
        continue

    if 'MatMul' in detail['name']:
        weights_arrays.append(tensor)
    elif 'BiasAdd' in detail['name']:
        biases_arrays.append(tensor)

# Combine all weights and biases into a single 3D and 2D array, respectively
weights_combined = format_sv_weights_array(weights_arrays)
biases_combined = format_sv_biases_array(biases_arrays)

# Prepare the content to be written to the file
content = ""
content += f"parameter LAYERS = {layers};\n"
content += f"parameter int LAYER_WIDTHS[LAYERS+1] = {{{', '.join(map(str, reversed(layer_widths)))}}};\n\n"
content += "parameter int WEIGHTS [][][] = \n"
content += weights_combined
content += ";\n\n"
content += "parameter int BIASES [][] = \n"
content += biases_combined
content += ";\n"

# Write the content to a file
with open(output_file_path, 'w') as file:
    file.write(content)

print(f"SystemVerilog parameters have been written to {output_file_path}")


Skipping tensor tfl.quantize due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/flatten/Reshape due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense/MatMul;sequential/dense/Relu;sequential/dense/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense_1/MatMul;sequential/dense_1/Relu;sequential/dense_1/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential/dense_2/MatMul;sequential/dense_2/Relu;sequential/dense_2/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
SystemVerilog parameters have been written to systemverilog_parameters.sv
